In [66]:
# all imports
import tensorflow_datasets as tfds 
import os 
import tensorflow as tf
import numpy as np 
import pandas as pd

In [1]:
# download the dataset

data = tfds.load(name='reddit')

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/reddit/1.0.0.incompleteVCH3YH/reddit-train.tfrecord


  0%|          | 0/3848330 [00:00<?, ? examples/s]

Dataset reddit downloaded and prepared to /root/tensorflow_datasets/reddit/1.0.0. Subsequent calls will reuse this data.


In [36]:
# so now data is in here
# the data sits on disk, and not memory
path = '/root/tensorflow_datasets/reddit/1.0.0/'
# os.listdir(path)
# Example data file 'reddit-train.tfrecord-00030-of-00256'

# change wd
os.chdir(path)

In [37]:
# have to specify the structure of the files. I found the FeaturesDict() structure 
# from the tensorflow page for this dataset: https://www.tensorflow.org/datasets/catalog/reddit. 
# this guide talks about reading it in: https://www.tensorflow.org/datasets/external_tfrecord. 
# but I never ended up needing this exactly. 
features = tfds.features.FeaturesDict({
    # these are the keys to each datafile
    'author': tf.string,
    'body': tf.string,
    'content': tf.string,
    'id': tf.string,
    'normalizedBody': tf.string,
    'subreddit': tf.string,
    'subreddit_id': tf.string,
    'summary': tf.string,
})

# Reading in the data in several ways

In [38]:
# read in a raw records.
# just drop in 1 example for now. 
filenames = ['reddit-train.tfrecord-00030-of-00256']
raw_dataset = tf.data.TFRecordDataset(filenames)
for raw_record in raw_dataset.take(10):
  print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b"\n\xf9=\n\xad\x14\n\x04body\x12\xa4\x14\n\xa1\x14\n\x9e\x14I just landed a new job making more than just $10/hr and I've decided I need to really bite the bullet and get a hold of my finances. I'm working on the first step in the 'How to handle $' link on the sidebar. The Excel spreadsheet I decided to use is [this one]( but I'm not married to it and can use another if that's recommended.\n\n\nMy paychecks right now are different every two weeks and I was able to change it to be able to accept a different amount for the first and second paycheck of the month. I don't think this'll be much of an issue though once the new job starts.\n\n\nI put in my monthly bills and their due dates, and it automatically puts me negative, whether or not they've been paid. In fact, there's no way for me to differentiate the ones that have been paid and the ones that haven't. I also put in the debts that I have. I want to put in the minimum payment and due date 

In [39]:
# Parsing it into a json format
filenames = ['reddit-train.tfrecord-00030-of-00256']
raw_dataset = tf.data.TFRecordDataset(filenames)

# limit 2 examples
i = 0
for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)
  i += 1
  if i > 2:
    break

features {
  feature {
    key: "author"
    value {
      bytes_list {
        value: "notsoltd"
      }
    }
  }
  feature {
    key: "body"
    value {
      bytes_list {
        value: "I just landed a new job making more than just $10/hr and I\'ve decided I need to really bite the bullet and get a hold of my finances. I\'m working on the first step in the \'How to handle $\' link on the sidebar. The Excel spreadsheet I decided to use is [this one]( but I\'m not married to it and can use another if that\'s recommended.\n\n\nMy paychecks right now are different every two weeks and I was able to change it to be able to accept a different amount for the first and second paycheck of the month. I don\'t think this\'ll be much of an issue though once the new job starts.\n\n\nI put in my monthly bills and their due dates, and it automatically puts me negative, whether or not they\'ve been paid. In fact, there\'s no way for me to differentiate the ones that have been paid and the ones tha

In [40]:
# parse the output by defining the var types
# this seems like the best way so far.
def map_fn(serialized_example):

  file_scruct = {
      'author': tf.io.FixedLenFeature([], tf.string),
      'body': tf.io.FixedLenFeature([], tf.string),
      'content': tf.io.FixedLenFeature([], tf.string),
      'id': tf.io.FixedLenFeature([], tf.string),
      'normalizedBody': tf.io.FixedLenFeature([], tf.string),
      'subreddit': tf.io.FixedLenFeature([], tf.string),
      'subreddit_id': tf.io.FixedLenFeature([], tf.string),
      'summary': tf.io.FixedLenFeature([], tf.string),
  }

  example1 = tf.io.parse_single_example(serialized_example, file_scruct)
  return example1



# just do 1-2 for now, but could put all file paths in here
all_files = tf.data.TFRecordDataset([path + 'reddit-train.tfrecord-00231-of-00256', path + 'reddit-train.tfrecord-00030-of-00256'])
dataset = all_files.map(map_fn)

# print it out
# this prints out too many
# for item in dataset:
#   print(item['author'], item['summary'], item['subreddit'])

# just do 10
i = 0 
for item in dataset:
  tup1 = (item['author'], item['body'], item['content'], item['summary'], item['subreddit'])
  print(tup1)
  i += 1
  if i > 10:
    break

(<tf.Tensor: shape=(), dtype=string, numpy=b'IllegalFtb'>, <tf.Tensor: shape=(), dtype=string, numpy=b"Don't worry, sounds reasonable to me. I post very minimally outside of short comments. I'll keep formatting and TL;DR in mind in the future.">, <tf.Tensor: shape=(), dtype=string, numpy=b"Don't worry, sounds reasonable to me. I post very minimally outside of short comments. I'll keep formatting and">, <tf.Tensor: shape=(), dtype=string, numpy=b'in mind in the future.'>, <tf.Tensor: shape=(), dtype=string, numpy=b'xboxone'>)
(<tf.Tensor: shape=(), dtype=string, numpy=b'AngryDelphiDev'>, <tf.Tensor: shape=(), dtype=string, numpy=b"Its not often I feel the need to chime in: but this is absolutely the wrong response. Group projects in a higher education setting are generally a complete waste of time. They don't teach you as much as doing (a smaller) assignment yourself and they don't teach you teamwork in the real world (where you have the ability to extend deadlines, reassign teammates a

In [50]:
# another way to read it in. Seems messy
all_files = tf.data.TFRecordDataset(['reddit-train.tfrecord-00231-of-00256', 'reddit-train.tfrecord-00030-of-00256'])
output = next(iter(all_files.batch(30000)))
output[0]

<tf.Tensor: shape=(), dtype=string, numpy=b"\n\xd4\x04\n\x9b\x01\n\x04body\x12\x92\x01\n\x8f\x01\n\x8c\x01Don't worry, sounds reasonable to me. I post very minimally outside of short comments. I'll keep formatting and TL;DR in mind in the future.\n%\n\x07summary\x12\x1a\n\x18\n\x16in mind in the future.\n\x1c\n\x0csubreddit_id\x12\x0c\n\n\n\x08t5_2xbci\n\x11\n\x02id\x12\x0b\n\t\n\x07cozsjsb\n\xa7\x01\n\x0enormalizedBody\x12\x94\x01\n\x91\x01\n\x8e\x01Don't worry, sounds reasonable to me. I post very minimally outside of short comments. I'll keep formatting and TL;DR in mind in the future. \n\n~\n\x07content\x12s\nq\noDon't worry, sounds reasonable to me. I post very minimally outside of short comments. I'll keep formatting and\n\x18\n\tsubreddit\x12\x0b\n\t\n\x07xboxone\n\x18\n\x06author\x12\x0e\n\x0c\n\nIllegalFtb">

In [52]:
# trying to get all the subreddits
all_files = tf.data.TFRecordDataset(['reddit-train.tfrecord-00231-of-00256', 'reddit-train.tfrecord-00030-of-00256'])
dataset = all_files.map(map_fn)
subreddits = []
for item in dataset: 
  subreddits.append(item['subreddit'])

In [58]:
print('Number of items {}'.format(len(subreddits)))
subreddits[:10]

Number of items 30065


[<tf.Tensor: shape=(), dtype=string, numpy=b'xboxone'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'funny'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'AskReddit'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'investing'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'kindle'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'seduction'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'atheism'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'funny'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'soccer'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'funny'>]

In [60]:
# can join them into 1 string
tf.strings.join(subreddits[:10], separator=' ', name='subreddits_in_1_string')

<tf.Tensor: shape=(), dtype=string, numpy=b'xboxone funny AskReddit investing kindle seduction atheism funny soccer funny'>

In [76]:
type(x[0])
tf.Variable(x[0]).map_fn(decode_string)

AttributeError: ignored

In [83]:
# def decode_string(string):
#   return string.decode('utf-8')
# tf.map_fn(fn = decode_string, elems = x[0])







array(['xboxone', 'funny', 'AskReddit', ..., 'TACPodcast', 'dragoncon',
       'KarmaCourt'], dtype='<U21')

In [84]:
x = tf.unique_with_counts(subreddits, out_idx=tf.dtypes.int32, name='subreddit_distinct_counts')
decode_string = np.vectorize(lambda x: x.decode('utf-8'))

subreddit_df = pd.DataFrame({
    "subreddit": decode_string(x[0].numpy()),
    "freq": x[2].numpy()
})
subreddit_df.sort_values('freq', ascending=False)

,subreddit,freq
2,AskReddit,4627
10,relationships,2662
13,leagueoflegends,849
38,trees,377
41,relationship_advice,377
...,...,...
1914,plugdj,1
1911,stalker,1
1910,OpenLetter,1
1908,orchestra,1
